In [1]:
# imports

In [2]:
import requests
from IPython.display import JSON

In [3]:
import pandas as pd
import os

# Foursquare

### Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [4]:
#For all bike stations in the city of your choice, get the latitude and longitude.
#Show example of what the first index of ll: list contains.
df = pd.read_csv("bike_info.csv")
column = list(df)
ll = []
for i in range(len(column)-1):
    ll.append((f"{str(df['latitude'][i])},{str(df['longitude'][i])}"))
print(ll[0])

43.665269,-79.319796


In [5]:
def station(llvalue):
    '''
    llvalue: str
    return: HTTPCode
    Take a string that looks like "latitude,longitude"
    '''
    headers = {"Accept": "application/json"}
    headers["Authorization"] = os.environ.get('FOURSQUARE_API_KEY')
    fields = 'name,rating,categories,price,popularity,geocodes'
    url = (f'https://api.foursquare.com/v3/places/search')
    parameters = {'ll' : llvalue,
                  'fields' : fields,
                  'radius' : 1000,
                  'categories' : '13065,13003',
                  'limit' : '50'
                 }
    four_square_networks = requests.get(url, headers=headers, params=parameters)
    return four_square_networks


In [6]:
JSON(station(ll[2]).json())

<IPython.core.display.JSON object>

### Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [7]:
def POI_fs(request):
    '''
    request: HTTP Code <200>
    Take in a HTTP code and convert it to JSON.
    Return: List
    '''
    request_in_json = request.json()
    results = request_in_json['results']
    list_of_results = []
    for result in results:
        new_dict = {}
        new_dict['name'] = result.get('name', None)
        new_dict['popularity'] = result.get('popularity', None)
        new_dict['price'] = result.get('price', None)
        new_dict['rating'] = result.get('rating', None)
        new_dict['categories_name'] = result.get('categories', None)[0].get('name', None)
        new_dict['latitude'] = result.get('geocodes', None).get('main', None).get('latitude', None)
        new_dict['longtitude'] = result.get('geocodes', None).get('main', None).get('longitude', None)
        list_of_results.append(new_dict)
    return list_of_results

### Put your parsed results into a DataFrame

In [8]:
def data_frame(listOfResults):
    '''
    listOfResults: list -> DataFrame
    '''
    return pd.DataFrame(listOfResults)

# Yelp

### Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [9]:
def yelp_station(ll_value):
    '''
    ll_value: str
    return: HTTPCode
    Take a string that looks like "latitude,longitude"
    '''
    headers_yelp = {}
    headers_yelp["Authorization"] = "Bearer " + os.environ.get('YELP_API_KEY')
    url = "https://api.yelp.com/v3/businesses/search"
    parameters = {'latitude' : ll_value.split(",")[0],
                  'longitude' : ll_value.split(",")[1],
                  'radius': 1000,
                  'categories' : "bars,restaurants",
                  'limit': 50
                 }
    yelp_networks = requests.get(url, headers=headers_yelp, params=parameters)
    return yelp_networks

In [10]:
JSON(test.json())

NameError: name 'test' is not defined

### Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
def POI_yelp(request):
    '''
    request: HTTP Code <200>
    Take in a HTTP code and convert it to JSON.
    Return: List
    '''
    request_in_json = request.json()
    businesses = request_in_json['businesses']
    list_of_businesses = []
    for business in businesses:
        new_dict = {}
        new_dict['name'] = business.get('name', None)
        new_dict['price'] = business.get('price', None)
        new_dict['rating'] = business.get('rating', None)
        new_dict['latitude'] = business.get('coordinates', None).get('latitude', None)
        new_dict['longtitude'] = business.get('coordinates', None).get('longitude', None)
        list_of_businesses.append(new_dict)
    return list_of_businesses

Put your parsed results into a DataFrame

# Create a Dataframe

In [ ]:
def df_yelp(poi_yelp):
    return pd.DataFrame(POI_yelp(test))

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

I think foursquare worked better for me, it had more data to choose from. I have also tried a different latitude and longitude from Greece prior to this observation in Toronto. While foursquare gave me a lot of data, there was no data at all from yelp. latitude:38.0855680022
longitude:23.9776389963

Get the top 10 restaurants according to their rating

In [ ]:
def mainfunction_foursq(llvalue):
    request_code = station(llvalue)
    point = POI_fs(request_code)
    return data_frame(point)

print(mainfunction_foursq("43.665269,-79.319796").sort_values(by='rating', ascending=False).head(10))

In [ ]:
def mainfunction_yelp(llvalue):
    request_code = yelp_station(llvalue)
    point = POI_yelp(request_code)
    return df_yelp(point)

print(mainfunction_yelp("43.665269,-79.319796").sort_values(by='rating', ascending=False).head(10))

### Joining data from Part 1 to 2 and loading it into a csv.

In [ ]:
list_of_df = []
for indx in range(len(ll)):
    list_of_df.append(mainfunction_foursq(ll[indx]))

newdfs = pd.concat(list_of_df)
newdfs.to_csv('fsrestaurant_info.csv')

In [ ]:
list_of_df_yelp = []
for indx in range(len(ll)):
    list_of_df_yelp.append(mainfunction_yelp(ll[indx]))

newdfs_yelp = pd.concat(list_of_df_yelp)
newdfs_yelp.to_csv('yelprestaurant_info.csv')